In [1]:
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()  
openai.api_key = os.getenv("OPENAI_API_KEY")

UNIQUE_KEY = "property_id"

In [2]:
def get_embedding(raw, wait=True):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful english teacher. From the user, you will be given a bad description of a house.Rewrite that bad description. Only use information given by the user.",
            },
            {"role": "user", "content": raw},
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    try:
        return response.choices[0]["message"]["content"]
    except KeyError:
        raise UserWarning("No response from GPT-3.5. Try again.")
        return None

In [3]:
get_embedding("Peacefully located away from the hustle and bustle of the main city, this House will give you the peace of mind you need most. Looking for a lucrative investment or a shelter? Consider the following buy. 2250 Square Feet unit options are limited, so hurry up before it's too late. The ideal place for family living made available at only Rs. 29,000,000. The location of the House in PGECHS Phase 2 is such that all the attractions and landmarks of the city are easily accessible from it. Grab the opportunity to own your own home in Lahore with affordable housing options available. The details of the property are listed down below. The lawn outside is large enough to let you host a party with family and friends. A large sitting room with windows like the one that comes with this House can elevate any property. The drawing room in this house lets you entertain your guests in a neat and beautiful space. A powder room is a great addition especially if you frequently entertain guests. The House has double glazed windows that are stylish yet practical. Our agents can help address your concerns, if you dial the given number.")

"Nestled in a serene location, far from the bustling city, this house provides the tranquility and peace of mind you desire. Whether you are seeking a profitable investment or a cozy dwelling, this property is worth considering. With limited 2250 square feet units available, time is of essence, so act quickly before it's too late. Priced at only Rs. 29,000,000, this house offers an ideal living space for families. Situated in the desirable neighborhood of PGECHS Phase 2, it provides easy access to all the city's attractions and landmarks. Take advantage of this opportunity to own an affordable home in Lahore. \n\nThe property features a spacious lawn where you can leisurely host gatherings with your loved ones. Furthermore, the house boasts a large sitting room with beautiful windows that add elegance and charm to the property. The inviting drawing room is the perfect space to entertain guests, showcasing a neat and beautiful ambiance. Additionally, the presence of a powder room is a c

In [4]:
df = pd.read_csv("small_df.csv", index_col=None)

In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
descriptions = {} 
BATCH_SIZE = 1000
with ThreadPoolExecutor(max_workers=10) as executor:
    for j in tqdm(range(0, len(df), 20)):
        batch_df = df.iloc[j:min(j+20, len(df))]
        future_to_key = {}
        
        # request this batch
        for i, row in enumerate(batch_df.iterrows()):
            index, data = row
            res = {executor.submit(get_embedding, data["description"]): data[UNIQUE_KEY]}
            future_to_key.update(res)

        # get responses for this batch
        for future in as_completed(future_to_key):
            key = future_to_key[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (key, exc))
            else:
                res = {key: data}
                descriptions.update(res)


with open('new-description.json', 'w') as fp:
    json.dump(descriptions, fp) 

  0%|          | 0/588 [00:00<?, ?it/s]

  1%|          | 3/588 [01:09<3:53:49, 23.98s/it]

In [9]:
12000/500

24.0